In [17]:
from pandas_datareader import data as pdr
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import cvxpy as cvx
from scipy.stats import norm
from numpy.linalg import inv
import math
import benchmarks

# for plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import matplotlib.pyplot as plt

from benchmarks import *

# Initialize plotly offline
plotly.offline.init_notebook_mode(connected=True)

## Load Data

### Hundred stocks

In [18]:
# Read in the data
hundred_stock_prices = pd.read_csv('./00_Data/hundred_stocks.csv', header=0, index_col = 0)

In [19]:
hundred_stock_prices.index = pd.to_datetime(hundred_stock_prices.index)

#Process Stock Data
# hundred_stock_relative_prices, hundred_stock_returns = process_stock_data(hundred_stock_prices)

#Read processed data to save time
hundred_stock_relative_prices = pd.read_csv('./00_Data/hundred_stock_relative_prices.csv', header=0, index_col = 0)
hundred_stock_returns = pd.read_csv('./00_Data/hundred_stock_returns.csv', header=0, index_col = 0)

#### SPY

In [20]:
# download_stock_data('2001-01-02', '2019-07-17', ['SPY'], 'SPY_same_dates_as_hundred_stocks')

In [31]:
# Read in the data
SPY_prices = pd.read_csv('./00_Data/SPY_same_dates_as_hundred_stocks.csv', header=0, index_col = 0)
SPY_prices.index = pd.to_datetime(hundred_stock_prices.index)

#Process Stock Data
# SPY_relative_prices, SPY_returns = process_stock_data(SPY_prices)

# Read in data to save time
SPY_relative_prices = pd.read_csv('./00_Data/SPY_relative_prices_2001to2019.csv', header=0, index_col = 0)
SPY_returns = pd.read_csv('./00_Data/SPY_returns_2001to2019.csv', header=0, index_col = 0)

## Risk Aware Portfolio Selection Algorithm
#### Bug: cannot initialize gamma as cp.Parameter

In [34]:
def risk_aware_portfolio(stock_returns, gamma, delta):
    """
    input:
    stock_return: returns for the stock data
    gamma: the confidence level
    delta: number of dates we use to estimate the density of returns
    
    output:
    portfolio: portfolio selected
    """
    u = cp.Variable((stock_returns.shape[1], 1))
    a = cp.Variable()
    #     gamma = cp.Parameter(gamma)

    constraints = [cp.sum(u) == 1, u >= 0]

    portfolio = []

    for t in range(2, len(stock_returns)):
        coefficient = 1 / ((delta + t - 1) * (1 - gamma))

        sliced = stock_returns.iloc[:delta]
        sliced = sliced.to_numpy()
        inner_part = cp.pos(sliced * (-u) - a)
        first_sum = cp.sum(inner_part)

        sliced2 = stock_returns.iloc[:t - 1]
        sliced2 = sliced2.to_numpy()
        inner_part2 = sliced2 * (-u) - a
        second_sum = cp.sum(cp.pos(inner_part2))

        obj = cp.Minimize(a + coefficient * (first_sum + second_sum))

        prob = cp.Problem(obj, constraints)
        prob.solve(solver='SCS')
        portfolio.append(u.value)

    return portfolio

## Bandit Algorithm

In [35]:
def bandit_portfolio(stock_returns):
    n_time, n_assets = stock_returns.shape # n_time >> n_assets in our case
    portfolio = np.zeros((n_assets, n_time)) # the output portfolio
    Rbar = np.zeros((n_assets,1)) # empirical mean of return for assets

    num_selected = {}
    for i in range(n_assets):
        num_selected[i] = 0
        
    # for loop up until n_time, output the bandit portfolio at each time t
    
    for t in range(n_time):
        
        if t < n_assets:
            portfolio[t, t] = 1
            Rbar[t] = stock_returns.iloc[t, t]
            num_selected[t] = 1
            continue
            
        max_asset = 0
        max_upper_bound = 0
        for asset in range(n_assets):
            
            avg_reward = Rbar[asset]
            right_part = np.sqrt(2*np.log(t) / num_selected[asset])
            upper_bound = avg_reward + right_part
            
            if upper_bound > max_upper_bound:
                max_asset = asset
                max_upper_bound = upper_bound
        portfolio[max_asset, t] = 1
        #pull
        Rbar[max_asset] = (Rbar[max_asset] * num_selected[max_asset] + stock_returns.iloc[t, max_asset]) / (num_selected[max_asset] + 1)
        num_selected[max_asset] += 1
        
    return portfolio

## Portfolio selection with original Bandit algorithm

In [36]:
og_bandit_selection = bandit_portfolio(hundred_stock_returns)

### Benchmarking
> **WARNING**: Takes long time to run!

In [54]:
benchmark_portfolio(og_bandit_selection,
                    'Original Bandit',
                    SPY_relative_prices.iloc[-100:],
                    stock_returns=hundred_stock_returns.iloc[-100:],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[-100:])

ValueError: Length of values does not match length of index